In [1]:
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import shutil
from label_pipeline import InteriorImageClassifier, generate_class_label2ref_images
from config import (
    MODEL_NAME,
    BASE_PROMPT_EN,
    BASE_PROMPT_RU,
    TRASH_FILTER_PROMPT_EN,
    TRASH_FILTER_PROMPT_RU,
    A0_ONLY_FILTER_PROMPT_EN,
    A0_ONLY_FILTER_PROMPT_RU,
    B1_ONLY_FILTER_PROMPT_EN,
    B1_ONLY_FILTER_PROMPT_RU,
    D0_ONLY_FILTER_PROMPT_EN,
    D0_ONLY_FILTER_PROMPT_RU,
    D1_ONLY_FILTER_PROMPT_EN,
    D1_ONLY_FILTER_PROMPT_RU
)

In [3]:
def process_directory(
        classifier: InteriorImageClassifier,
        image_dir: Path,
        output_dir: Path,
        ref_images_dir: Path | None = None,
        extensions: tuple = (".jpg", ".png", ".jpeg")
    ) -> None:
    """
    Обрабатывает изображения и сортирует по папкам классов
    """
    output_dir.mkdir(parents=True, exist_ok=True)  # Создаем корневую директорию для результатов
    
    # Обрабатываем каждое изображение
    img_files = [f for f in image_dir.iterdir() if f.suffix.lower() in extensions]
    for img_file in tqdm(img_files, desc="Обработка изображений"):
        class_label2ref_images = None
        if ref_images_dir is not None:
            class_label2ref_images = generate_class_label2ref_images(
                ref_images_dirpath=ref_images_dir,
                max_files_per_class=1
            )
        image = Image.open(img_file)
        result = classifier(image, class_label2ref_images=class_label2ref_images)
        
        class_dir = output_dir / result["class"]  # Создаем папку класса (A0, A1... ERROR)
        class_dir.mkdir(exist_ok=True)
        
        shutil.copy2(img_file, class_dir / img_file.name)  # Копируем изображение в папку класса
    
    print(f"\nГотово! Результаты в {output_dir}")


In [4]:
def process_directory2_for_filtering(
        classifier: InteriorImageClassifier,
        image_dir: Path,
        output_dir: Path,
        custom_prompt: str| None = None,
        ref_images_dir: Path | None = None,
        extensions: tuple = (".jpg", ".png", ".jpeg")
    ) -> None:
    """
    Обрабатывает изображения и сортирует по папкам классов
    """
    output_dir.mkdir(parents=True, exist_ok=True)  # Создаем корневую директорию для результатов
    
    # Обрабатываем каждое изображение
    img_files = [f for f in image_dir.iterdir() if f.suffix.lower() in extensions]
    for img_file in tqdm(img_files, desc="Обработка изображений"):
        class_label2ref_images = None
        if ref_images_dir is not None:
            class_label2ref_images = generate_class_label2ref_images(
                ref_images_dirpath=ref_images_dir,
                max_files_per_class=1
            )
        image = Image.open(img_file)
        result = classifier(image, custom_prompt=custom_prompt)
        
        if result["class"] != "D0":  # trash
            shutil.move(img_file, output_dir)
        else:
            raw_response = result['raw_response'].split()
            if len(raw_response) >= 2:
                confidence = raw_response[1]
                if confidence.isdigit() and int(confidence) <= 8:
                    shutil.move(img_file, output_dir)
    
    print(f"\nГотово! Результаты в {output_dir}")

In [2]:
# Инициализация классификатора
classifier = InteriorImageClassifier(model_name=MODEL_NAME, base_prompt=BASE_PROMPT_EN)
print(classifier.pipe.torch_dtype)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
Device set to use cuda:0


torch.bfloat16


In [ ]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"

image_dir = data_dir / "cian_data/D0"
output_dir = data_dir / "cian_data/D0_uncertain"

process_directory2_for_filtering(
    classifier=classifier,
    image_dir=image_dir,
    output_dir=output_dir,
    custom_prompt=D0_ONLY_FILTER_PROMPT_EN
)

In [ ]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"
ref_images_dir = data_dir / "reference_images/resized_448x448/A0"
print(ref_images_dir)

class_label2ref_images = generate_class_label2ref_images(
    ref_images_dirpath=ref_images_dir,
    max_files_per_class=1
)
class_label2ref_images

In [ ]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"

image_dir = data_dir / "cosmetic/cosmetic-"
output_dir = data_dir / "labeled_with_few_shot"
ref_images_dir = data_dir / "reference_images_resized"

process_directory(
    classifier=classifier,
    image_dir=image_dir,
    output_dir=output_dir,
    ref_images_dir=ref_images_dir
)

In [4]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"
cian_data_dir = data_dir / "cian_data"
cian_data_dir_B1 = cian_data_dir / "B1"
output_dir = cian_data_dir / "B1_uncertain"
output_dir.mkdir(exist_ok=True)

ref_images_dir = data_dir / "reference_images" / "resized_448x448"
ref_images_dir_B1 = ref_images_dir / "B1"
max_examples_per_class = 10

class_label2ref_images = {'B1': []}
for p in ref_images_dir_B1.iterdir():
    class_label2ref_images['B1'].append(Image.open(p))


# Начало промпта
content = [
    {"type": "text", "text": B1_ONLY_FILTER_PROMPT_EN},
    {"type": "text", "text": "\nReference examples:"}
]


for class_label, images in class_label2ref_images.items():
    if images:
        selected_images = images[:max_examples_per_class]
        # Добавляем в контент
        for image in selected_images:
            content.extend([
                {"type": "image", "image": image},
                {"type": "text", "text": f"Class: {class_label}"}
            ])

print(cian_data_dir_B1)
moved = 0
extensions = (".jpg", ".png", ".jpeg")
for img_path in tqdm(sorted(cian_data_dir_B1.iterdir())):
    if img_path.suffix.lower() not in extensions:
        continue
    
    # Добавляем целевое изображение
    target_content = [
        {"type": "text", "text": "\nNow classify THIS image:"},
        {"type": "image", "image": Image.open(img_path)},
        {"type": "text", "text": "Answer ONLY with class label:"}
    ]
    messages = [{"role": "user", "content": content + target_content}]
    raw_response = classifier._get_model_response(inputs=messages, max_new_tokens=10)
    predicted_class = classifier.parse_model_response(raw_response)

    if predicted_class != "B1":  # trash
        shutil.move(img_path, output_dir)
        moved += 1
    else:
        raw_response = raw_response.split()
        if len(raw_response) >= 2:
            confidence = raw_response[1]
            if confidence.isdigit() and int(confidence) <= 8:
                shutil.move(img_path, output_dir)
                moved += 1
print(f"Total moved: {moved}")

Директория: /home/little-garden/CodeProjects/InteriorClass
/home/little-garden/CodeProjects/InteriorClass/data/cian_data/B1


100%|██████████| 71335/71335 [24:59:23<00:00,  1.26s/it]   

Total moved: 47700


In [ ]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"
cian_data_dir = data_dir / "cian_data"
cian_data_dir_D1 = cian_data_dir / "D1"
output_dir = cian_data_dir / "D1_uncertain"

ref_images_dir = data_dir / "reference_images" / "resized_448x448"
ref_images_dir_D1 = ref_images_dir / "D1"
max_examples_per_class = 10

class_label2ref_images = {'D1': []}
for p in ref_images_dir_D1.iterdir():
    class_label2ref_images['D1'].append(Image.open(p))


# Начало промпта
content = [
    {"type": "text", "text": D1_ONLY_FILTER_PROMPT_EN},
    {"type": "text", "text": "\nReference examples:"}
]


for class_label, images in class_label2ref_images.items():
    if images:
        selected_images = images[:max_examples_per_class]
        # Добавляем в контент
        for image in selected_images:
            content.extend([
                {"type": "image", "image": image},
                {"type": "text", "text": f"Class: {class_label}"}
            ])

print(cian_data_dir_D1)
moved = 0
for img_path in tqdm(sorted(cian_data_dir_D1.iterdir())):
    # Добавляем целевое изображение
    target_content = [
        {"type": "text", "text": "\nNow classify THIS image:"},
        {"type": "image", "image": Image.open(img_path)},
        {"type": "text", "text": "Answer ONLY with class label:"}
    ]
    messages = [{"role": "user", "content": content + target_content}]
    raw_response = classifier._get_model_response(inputs=messages, max_new_tokens=10)
    predicted_class = classifier.parse_model_response(raw_response)

    if predicted_class != "D1":  # trash
        shutil.move(img_path, output_dir)
        moved += 1
    else:
        raw_response = raw_response.split()
        if len(raw_response) >= 2:
            confidence = raw_response[1]
            if confidence.isdigit() and int(confidence) <= 8:
                shutil.move(img_path, output_dir)
                moved += 1
print(f"Total moved: {moved}")